In [1]:
%load_ext autoreload
%autoreload 

In [3]:
import sys
import numpy as np
import modin as pd
import plotly.express as px
from utils import get_df

path = "ray/criteo/optimizations/run_and_report_2024-03-11_01-15-59"
df = get_df(path)

<generator object Path.glob at 0x7ff986ac3140>
/mydata/adsprivate/systemx/logs/ray/criteo/optimizations/run_and_report_2024-03-11_01-15-59/run_and_report_d41c3_00000_0_initial_budget=5,optimization=0_2024-03-11_01-15-59/result.json
/mydata/adsprivate/systemx/logs/ray/criteo/optimizations/run_and_report_2024-03-11_01-15-59/run_and_report_d41c3_00001_1_initial_budget=5,optimization=1_2024-03-11_01-15-59/result.json
/mydata/adsprivate/systemx/logs/ray/criteo/optimizations/run_and_report_2024-03-11_01-15-59/run_and_report_d41c3_00002_2_initial_budget=5,optimization=2_2024-03-11_01-15-59/result.json


,log,initial_budget,optimization,dataset,config,timestamp,checkpoint_dir_name,done,training_iteration,trial_id,date,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore
0,[{'925829CF82DFDC74DE3F88F003CE7BEF': {'0': 4....,5,0,criteo,"{'sensitivity_metric': 'L1', 'user': {'sensiti...",1710140553,None,False,1,d41c3_00000,2024-03-11_02-02-33,2792.254262,2792.254262,2702193,node0.kelkost-190545.scaling-python-pg0.wisc.c...,128.105.146.74,2792.254262,1
1,[{'925829CF82DFDC74DE3F88F003CE7BEF': {'0': 4....,5,1,criteo,"{'sensitivity_metric': 'L1', 'user': {'sensiti...",1710140590,None,False,1,d41c3_00001,2024-03-11_02-03-10,2829.544390,2829.544390,2702194,node0.kelkost-190545.scaling-python-pg0.wisc.c...,128.105.146.74,2829.544390,1
2,[{'925829CF82DFDC74DE3F88F003CE7BEF': {'0': 4....,5,2,criteo,"{'sensitivity_metric': 'L1', 'user': {'sensiti...",1710140209,None,False,1,d41c3_00002,2024-03-11_01-56-49,2448.650684,2448.650684,2702211,node0.kelkost-190545.scaling-python-pg0.wisc.c...,128.105.146.74,2448.650684,1


In [68]:
df["log"][0]



[{'925829CF82DFDC74DE3F88F003CE7BEF': {'0': 4.99,
   '1': 4.99,
   '2': 4.99,
   '3': 4.99,
   '4': 4.99,
   '5': 4.99,
   '6': 4.99,
   '7': 4.99,
   '8': 4.99,
   '9': 4.99,
   '10': 4.99,
   '11': 4.99}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'5DC3ED9F13993DF6471EBE25BD88BD24': {'0': 4.99}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'6BFFDE668C5BC3938C875D3D0940E699': {'0': 4.98}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'9D8E4D89210B822B2060852D2DD143FE': {'0': 4.99,
   '1': 4.99,
   '2': 4.99,
   '3': 4.99,
   '4': 4.99,
   '5': 4.99,
   '6': 4.99,
   '7': 4.99,
   '8': 4.99,
   '9': 4.99,
   '10': 4.99,
   '11': 4.99,
   '12': 4.99,
   '13': 4.99,
   '14': 4.99,
   '15': 4.99,
   '16': 4.99,
   '17': 4.99,
   '18': 4.99,
   '19': 4.99,
   '20': 4.99}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'6BFFDE668C5BC3938C875D3D0940E699': {'0': 4.99, '1': 4.99}},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'C13DEAF04388B219AC0

In [26]:
def foo(bar):
    experimets = []
    for user_log in bar:
        budget_list = []
        for adv_dict in user_log:
            for epoch_budgets in adv_dict.values():
                budgets = [5] * 110
                for epoch, budget in epoch_budgets.items():
                    budgets[int(epoch)] = budget
                budget_list.append(budgets)
        experimets.append(budget_list)
    return experimets

exps = foo(df['log'])

In [35]:
len(exps[0][0])

n = []
for exp in exps:
    # n.append(np.array(exp))
    print(np.mean(np.array(exp), axis=1))

[4.99890909 4.99990909 4.99981818 ... 4.99718182 4.99718182 4.99718182]
[4.99890909 4.99999248 4.99999693 ... 4.99718182 4.99718182 4.99718182]
[4.99990909 4.99999248 4.99999693 ... 4.99990909 4.99990909 4.99990909]


In [37]:
d = {"1": 4, "2": 4, "3": 0}

In [38]:
d

{'1': 4, '2': 4, '3': 0}

In [47]:
import pandas as pdd

d = {"1": 4, "2": 4, "9": 4}

# Initialize a dense vector of zeros
dense_vector = np.ones(10)*1

# Convert keys to integers and update dense vector
keys = np.array(list(map(int, d.keys())))
print(keys)
values = np.array(list(d.values()))
print(values)
dense_vector[keys - 1] = values

# pdd.DataFrame(d)
dense_vector

[1 2 9]
[4 4 4]


array([4., 4., 1., 1., 1., 1., 1., 1., 4., 1.])

In [71]:
# b = df["remaining_budgets_per_epoch"]
# initial_budget = df["initial_budget"]


b = [{"1": 4, "2": 4, "9": 0},
         {"2": 2, "3": 5, "5": 3},
         {"4": 1, "6": 2, "8": 4}]
initial_budget = 5

max_index = max(max(map(int, d.keys())) for d in b)
dense_matrix = np.ones((len(b), max_index + 1)) * initial_budget
for i, d in enumerate(b):
    keys = np.array(list(map(int, d.keys())))
    values = np.array(list(d.values()))
    dense_matrix[i, keys] = values
    
sum_vector = np.mean(dense_matrix, axis=0)
print(sum_vector)

[5.         4.66666667 3.66666667 5.         3.66666667 4.33333333
 4.         5.         4.66666667 3.33333333]
